In [1]:
import numpy as np
import pandas as pd

In [4]:
# load the data, 
columns = ['time',
 'distance',
 'easting',
 'northing',
 'ellipsoid height',
 'latitude',
 'longitude',
 'ellipsoid height_1',
 'roll',
 'pitch',
 'heading',
 'east velocity',
 'north velocity',
 'up velocity',
 'east sd',
 'north sd',
 'height sd',
 'roll sd',
 'pitch sd',
 'heading sd']
imu_df = pd.read_csv("../../Data/imu_data/export_Mission_3.txt", sep='[ ]+',engine='python',skiprows=28,header=None,names=columns)

In [5]:
imu_df.head()

,time,distance,easting,northing,ellipsoid height,latitude,longitude,ellipsoid height_1,roll,pitch,heading,east velocity,north velocity,up velocity,east sd,north sd,height sd,roll sd,pitch sd,heading sd
0,588578.00495,0.000,563627.452,4496423.282,-12.035,40.616185,-74.247795,-12.035,-1.519,-3.905,258.653,-27.776,-7.096,1.765,0.067,0.075,0.103,0.022,0.022,0.055
1,588578.00995,0.144,563627.314,4496423.245,-12.027,40.616185,-74.247796,-12.027,-1.507,-3.901,258.639,-27.779,-7.095,1.761,0.067,0.075,0.103,0.022,0.022,0.055
2,588578.01495,5.078,563632.107,4496424.399,-12.175,40.616195,-74.247740,-12.175,-1.491,-3.895,258.624,-27.783,-7.097,1.760,0.067,0.075,0.103,0.022,0.022,0.055
3,588578.01995,5.221,563631.969,4496424.362,-12.166,40.616194,-74.247741,-12.166,-1.473,-3.888,258.611,-27.784,-7.099,1.759,0.067,0.075,0.103,0.022,0.022,0.055
4,588578.02495,5.365,563631.830,4496424.325,-12.157,40.616194,-74.247743,-12.157,-1.455,-3.882,258.600,-27.786,-7.100,1.757,0.067,0.075,0.103,0.022,0.022,0.055
